In [ ]:
import pytraj as pt
import pandas as pd 
import mdtraj as md
import numpy as np
import glob
import matplotlib.pylab as plt 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pytraj as pt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.cm as cm
from scipy.spatial import distance

#run dependency script
%run "traj_analysis.py"

#specify dirs
TRAJHOME = "/home/jrocha/work_dir/mutations_g6pd/"
OUTPUTHOME = "/home/jrocha/work_dir/mutations_g6pd/"
common_ref = md.load_pdb('G6PD_WT_p_vacminned.pdb')

#specify trajectory dataset as produced by cppTraj (to combine)

wt = TRAJHOME+'WT_p/production_md/g6pd_WT_p_traj_510ns.nc'
MEDI = TRAJHOME+'MEDI_p/production_md/g6pd_MEDI_p_traj_510ns.nc'
ABEN = TRAJHOME+'Africa_A_ben/production_md/g6pd_ABEN_traj_510ns.nc'
ADEL = TRAJHOME+'Africa_A_del/production_md/g6pd_ADEL_traj_510ns.nc'
D350H = TRAJHOME+'D350H/production_md/g6pd_D350H_traj_510ns.nc'
M207T = TRAJHOME+'M207T/production_md/g6pd_M207T_traj_510ns.nc'
M212V = TRAJHOME+'M212V/production_md/g6pd_M212V_traj_510ns.nc'
R104H = TRAJHOME+'R104H/production_md/g6pd_R104H_traj_510ns.nc'
E156K = TRAJHOME+'E156K/production_md/g6pd_E156K_traj_510ns.nc'
L323P = TRAJHOME+'L323P/production_md/g6pd_L323P_traj_510ns.nc'

TOPWT=TRAJHOME+"WT_p/production_md/stripped.G6PD_WT_p_startMD.prmtop"
TOPMEDI=TRAJHOME+"MEDI_p/production_md/stripped.G6PD_MEDI_p_startMD.prmtop"
TOPABEN=TRAJHOME+"Africa_A_ben/production_md/stripped.G6PD_ABEN_startMD.prmtop"
TOPADEL=TRAJHOME+"Africa_A_del/production_md/stripped.G6PD_ADEL_startMD.prmtop"
TOPD350H = TRAJHOME+'D350H/production_md/stripped.G6PD_D350H_startMD.prmtop'
TOPM207T = TRAJHOME+'M207T/production_md/stripped.G6PD_M207T_startMD.prmtop'
TOPM212V = TRAJHOME+'M212V/production_md/stripped.G6PD_M212V_startMD.prmtop'
TOPR104H = TRAJHOME+'R104H/production_md/stripped.G6PD_R104H_startMD.prmtop'
TOPE156K = TRAJHOME+'E156K/production_md/stripped.G6PD_E156K_startMD.prmtop'
TOPL323P = TRAJHOME+'L323P/production_md/stripped.G6PD_L323P_startMD.prmtop'

traj_paths = [wt, MEDI, ABEN, ADEL, D350H, M207T, M212V, R104H, E156K, L323P ]
top_files = [TOPWT, TOPMEDI, TOPABEN, TOPADEL, TOPD350H, TOPM207T,  TOPM212V, TOPR104H, TOPE156K, TOPL323P ]
labels = ["WT", "MEDI", "ABEN","ADEL", "D350H", "M207T",  "M212V", "R104H", "E156K", "L323P" ]

traj_wt = md.load(wt, top=TOPWT)





In [ ]:
#select the start point for rmsf, its the a point just after equilbration has taken place 
#start = 30987
start = 30987

#run the RMSF for the trajectories
for trajectory,topology,label in  zip(traj_paths, top_files, labels  ) :
    mytraj = md.load(trajectory, top=topology)
    wt_and_vars_trajs = [traj_wt, mytraj]
    rmsf = RmsfCalculation(wt_and_vars_trajs, common_ref, start=start, offset = 25, labels = ["Ref", label] )
    rmsf.CalculateRmsf()
    rmsf.dumpToCsv(output=OUTPUTHOME+"/data/"+label+"_RMSF.csv")
    del rmsf

In [ ]:
#generate the rmsf plots
data_home = "/home/jrocha/work_dir/mutations_g6pd/data/"
plt.rcParams["figure.figsize"] = (14,32)


rmsfs = pd.read_csv(data_home+labels[0]+'_RMSF.csv')

for label in labels[1:]: 
    df = pd.read_csv(data_home+label+'_RMSF.csv')
    rmsfs=pd.concat([rmsfs, df[label]], axis=1)
rmsfs.to_csv(data_home+"all_RMSFs.csv", index=False)
plot_rmsf = RmsfPlots(data_home+"all_RMSFs.csv", 10)
plot_rmsf.PlotRmsfVars(range_of_data=[36, 970], output="testrmsf.pdf")

#plt.savefig("rmsf_All10_510ns.pdf")
plt.show()

In [ ]:
plt.savefig("rmsf_All10_510ns.pdf")
